# Real-World Applications of ML in Signal Processing: Long Short-Term Memory (LSTM) using TensorFlow

In this notebook, we will explore how to use machine learning, specifically TensorFlow, for time series prediction using Long Short-Term Memory (LSTM) networks. This notebook is designed for beginners with basic knowledge of Python and machine learning concepts.

## Objectives
1. Understand the basics of time series data.
2. Preprocess time series data for machine learning.
3. Build and train an LSTM network using TensorFlow.
4. Evaluate the model's performance.


## Step 1: Understanding Time Series Data

Time series data is a sequence of data points collected or recorded at specific time intervals. Examples of time series data include stock prices, weather data, and sensor readings.

Key concepts include:
- **Trend**: The overall direction in which the data is moving over a long period.
- **Seasonality**: Regular, repeating patterns in the data, often tied to calendar cycles.
- **Noise**: Random variations in the data that cannot be explained by the model.


## Step 2: Loading and Preprocessing Time Series Data

We'll use the `pandas` library to load and preprocess time series data. For this example, we'll use a dataset of daily minimum temperatures in Melbourne, Australia.

First, let's install the required libraries.


In [1]:
!pip install pandas matplotlib tensorflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data = pd.read_csv(url, parse_dates=['Date'], index_col='Date')

# Display the first few rows of the dataset
data.head()


Next, we'll visualize the time series data to understand its patterns and trends.

In [2]:
# Plot the time series data
plt.figure(figsize=(10, 6))
plt.plot(data, label='Daily Minimum Temperatures')
plt.title('Daily Minimum Temperatures in Melbourne')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.show()


## Step 3: Preparing the Data for LSTM

LSTM networks require the input data to be in a specific format. We'll split the data into training and testing sets, normalize it, and reshape it to be suitable for LSTM input.

In [3]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# Split the data into training and testing sets
train_size = int(len(data_scaled) * 0.8)
train, test = data_scaled[:train_size], data_scaled[train_size:]

# Function to create dataset with look_back
def create_dataset(dataset, look_back=1):
    X, y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), 0])
        y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(y)

look_back = 10
X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], look_back, 1))
X_test = np.reshape(X_test, (X_test.shape[0], look_back, 1))


## Step 4: Building and Training the LSTM Model

We'll build a simple LSTM model using TensorFlow and train it on our prepared dataset.

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Build the LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(look_back, 1)),
    LSTM(50),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


## Step 5: Evaluating the Model

We'll evaluate the model's performance by comparing its predictions to the actual values in the test set.

In [5]:
# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invert predictions
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train_actual = scaler.inverse_transform([y_train])
y_test_actual = scaler.inverse_transform([y_test])

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(data.index[:train_size + look_back], y_train_actual[0], label='Actual Training Data')
plt.plot(data.index[train_size + look_back:], y_test_actual[0], label='Actual Testing Data')
plt.plot(data.index[:train_size + look_back], train_predict, label='Training Predictions')
plt.plot(data.index[train_size + look_back:], test_predict, label='Testing Predictions')
plt.title('LSTM Model Predictions')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.show()


## Conclusion

In this notebook, we covered the basics of time series data, preprocessing it for machine learning, building and training an LSTM model using TensorFlow, and evaluating the model's performance. This is just a starting point, and there are many more advanced techniques and models to explore in the field of time series analysis and machine learning.